Adapted from *Python Machine Learning 2nd Edition* by [Sebastian Raschka](https://sebastianraschka.com), Packt Publishing Ltd. 2017

Code Repository: https://github.com/rasbt/python-machine-learning-book-2nd-edition

Code License: [MIT License](https://github.com/rasbt/python-machine-learning-book-2nd-edition/blob/master/LICENSE.txt)

## Applying Machine Learning To Sentiment Analysis

### Obtaining the IMDb movie review dataset

The IMDB movie review set can be downloaded from [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).

In [4]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(5)

,review,sentiment
0,at a Saturday matinee in my home town. I went ...,0
1,I love this movie. It is the first film Master...,1
2,"In the voice over which begins the film, Hughi...",1
3,"!!! Spoiler alert!!!<br /><br />The point is, ...",0
4,"This is an excellent film. No, it's not Mel Gi...",1


In [5]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [6]:
df.shape

(50000, 2)

### bag-of-words model

Transforming documents into feature vectors

By calling the fit_transform method on CountVectorizer, we can construct the vocabulary of the bag-of-words model and transformed the following three sentences into sparse feature vectors:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = vectorizer.fit_transform(docs)

Now let us print the contents of the vocabulary: 

In [8]:
print(vectorizer.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [9]:
print(vectorizer.get_feature_names())

['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather']


As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary, which maps the unique words that are mapped to integer indices. 

Next let us print the feature vectors that we just created. Each index position in the feature vectors corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. 

In [10]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Those values in the feature vectors are also called the raw **term frequencies**: *tf (t,d)*—the number of times a term t occurs in a document *d*.

### Assessing word relevancy via term frequency-inverse document frequency

In [11]:
np.set_printoptions(precision=2)

When we are analyzing text data, we often encounter words that occur across multiple documents from both classes. Those frequently occurring words typically don't contain useful or discriminatory information. A useful technique called term frequency-inverse document frequency (tf-idf) can be used to downweight those frequently occurring words in the feature vectors. The tf-idf can be defined as the product of the term frequency and the inverse document frequency:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

The inverse document frequency *idf(t, d)* can be calculated as:

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

where $n_d$ is the total number of documents, and *df(d, t)* is the number of documents *d* that contain the term *t*. 

Scikit-learn implements yet another transformer, the `TfidfTransformer`, that takes the raw term frequencies from `CountVectorizer` as input and transforms them into tf-idfs:

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
#print(tfidf.fit_transform(vectorizer.fit_transform(docs)).toarray())
print(tfidf.fit_transform(bag).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


As we saw in the previous subsection, the word is had the largest term frequency in the 3rd document, being the most frequently occurring word. However, after transforming the same feature vector into tf-idfs, we see that the word is is
now associated with a relatively small tf-idf (0.45) in document 3 since it is
also contained in documents 1 and 2 and thus is unlikely to contain any useful, discriminatory information.


Note: The equations for the idf and tf-idf that were implemented in scikit-learn `TfidfTransformer` are:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

and with the L2-normalization applied.

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2, 3.0^2, 3.39^2, 1.29^2, 1.29^2, 1.29^2, 2.0^2 , 1.69^2, 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

### Cleaning text data

We need to clean up the data by removing unwanted characters. 

In [13]:
df.loc[0, 'review']

"at a Saturday matinee in my home town. I went with an older friend (he was about 12) and my mom let me go because she thought the film would be OK (it's rated G). I was assaulted by loud music, STRANGE images, no plot and a stubborn refusal to make ANY sense. We left halfway through because we were bored, frustrated and our ears hurt. <br /><br />I saw it 22 years later in a revival theatre. My opinion had changed--it's even WORSE! Basically everything I hated about it was still there and the film was VERY 60s...and has dated badly. I got all the little in-jokes...too bad they weren't funny. The constant shifts in tone got quickly annoying and there's absolutely nothing to get a firm grip on. Some people will love this. I found it frustrating...by the end of the film I felt like throwing something heavy at the screen.<br /><br />Also, all the Monkees songs in this movie SUCK (and I DO like them).<br /><br />For ex-hippies only...or if you're stoned. I give this a 1."

We will remove all the HTML markups. For simplicity, we will now remove all punctuation marks except for emoticon characters since they are useful for sentiment analysis. Next, all non-word characters are removed and the text is converted to lowercase. 

In [14]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [15]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [16]:
preprocessor(df.loc[0, 'review'])

'at a saturday matinee in my home town i went with an older friend he was about 12 and my mom let me go because she thought the film would be ok it s rated g i was assaulted by loud music strange images no plot and a stubborn refusal to make any sense we left halfway through because we were bored frustrated and our ears hurt i saw it 22 years later in a revival theatre my opinion had changed it s even worse basically everything i hated about it was still there and the film was very 60s and has dated badly i got all the little in jokes too bad they weren t funny the constant shifts in tone got quickly annoying and there s absolutely nothing to get a firm grip on some people will love this i found it frustrating by the end of the film i felt like throwing something heavy at the screen also all the monkees songs in this movie suck and i do like them for ex hippies only or if you re stoned i give this a 1 '

Apply the preprocessor to movie review data. 

In [17]:
df['review'] = df['review'].apply(preprocessor)

In [18]:
df.head(5)

,review,sentiment
11841,i finally managed to sit through a whole episo...,0
19602,just what the world needed another superficial...,0
45519,i have to say despite it s reviews angels in t...,1
25747,only the chosen ones will appreciate the quali...,1
42642,anne natalie portman tells us about how much s...,1


### Processing documents into tokens

One simple way to tokenize documents is to split them into individual words by splitting documents by the whitespace characters. Another useful approach is word stemming, which is the process of transforming a word into its root form. Porter Stemmmer algorithm is one original stemming algorithm developed by Martin Porter in 1979. 

In [19]:
#import Natural Lanuage Toolkit 
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [20]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [21]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

We will also apply another useful technique called steop-word removal. 

In [22]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kbooth/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

### Training a logistic regression model for document classification

Prepare training set and the test set. 

In [46]:
X_train = df.iloc[:40000, 0].values
y_train = df.iloc[:40000, 1].values
X_test = df.iloc[40000:50000, 0].values
y_test = df.iloc[40000:50000, 1].values

Create a pipeline for training

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

lr_tfidf_pipeline = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=42, solver='lbfgs'))])

In [48]:
lr_tfidf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [49]:
lr_tfidf_pipeline.predict(X_test)

array([1, 0, 1, ..., 0, 0, 1])

In [50]:
print('Test Accuracy: %.3f' % lr_tfidf_pipeline.score(X_test, y_test))

Test Accuracy: 0.899


Try it out with a single example

In [51]:
label = {0: 'negative', 1: 'positive'}
example = ['The movie is so so']

print('Prediction: %s\nProbability: %.2f%%' %
      (label[lr_tfidf_pipeline.predict(example)[0]],
       np.max(lr_tfidf_pipeline.predict_proba(example)) * 100))

Prediction: negative
Probability: 58.31%


Tune the hyperparameters with GridSearchCV

In [52]:
from sklearn.model_selection import GridSearchCV


param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__C': [1.0, 10.0, 100.0]},
              ]


gs_lr_tfidf = GridSearchCV(lr_tfidf_pipeline, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [53]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
/Users/kbooth/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.5min finished
/Users/kbooth/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [54]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer at 0x7f83b9e67ea0>} 
CV Accuracy: 0.900


In [55]:
clf = gs_lr_tfidf.best_estimator_

In [56]:
clf.predict(X_test)

array([1, 0, 1, ..., 0, 0, 1])

In [57]:
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.906


Pickle the model

In [58]:
import pickle
file_pickle = open('test.pkl', 'wb')
pickle.dump(clf, file_pickle)
file_pickle.close()